In [1]:
#OpenCV module
import cv2
#os module for reading training data directories and paths
import os
#numpy to convert python lists to numpy arrays as it is needed by OpenCV face recognizers
import numpy as np


In [2]:
#there is no label 0 in our training data so subject name for index/label 0 is empty
subjects = [" ","Dileep","Yatisha"]


In [18]:
#function to detect face using OpenCV
def detect_face(img):
    #convert the test image to gray image as opencv face detector expects gray images
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #load OpenCV face detector, LBP 
    face_cascade = cv2.CascadeClassifier('hascascades/lbpcascade_frontalface.xml')
    #let's detect multiscale (some images may be closer to camera than others) images
    #result is a list of faces
    faces = face_cascade.detectMultiScale(gray, 
                    scaleFactor=1.2, minNeighbors=5)
    #if no faces are detected then return original img
    if (len(faces) == 0):
        return None, None
    #under the assumption that there will be only one face,'''
    #extract the face area
    (x, y, w, h) = faces[0]
    #return only the face part of the image
    return gray[y:y+w, x:x+h], faces[0]


In [4]:
def prepare_training_data(data_folder_path):
    #get the directories (one dir for each subject) in data folder
    dirs = os.listdir(data_folder_path)
    faces = [] #list to hold all subject faces
    labels = [] #list to hold labels for all subjects
    #let's go through each directory and read images within it
    for dir_name in dirs:
        #our subject dir start with letter 's' so
        #ignore any non-relevant directories if any
        if not dir_name.startswith("s"):
            continue;
            
        #extract label number of subject from dir_name
        #format of dir name = slabel
        #, so removing letter 's' from dir_name will give us label
        label = int(dir_name.replace("s", ""))
        #build path of directory containing images
        #sample subject_dir_path = "training-data/s1"
        subject_dir_path = data_folder_path + "/" + dir_name
        
        #get the images names that are inside the given subject directory
        subject_images_names = os.listdir(subject_dir_path)
        #go through each image name, read image, 
        #detect face and add face to list of faces
        for image_name in subject_images_names:
            #ignore system files like .DS_Store
            if image_name.startswith("."):
                continue;
            #build image path
            #sample image path = training-data/s1/1.png
            image_path = subject_dir_path + "/" + image_name
            #read image
            image = cv2.imread(image_path)
            #detect face
            face, rect = detect_face(image)
            #we will ignore faces that are not detected
            if face is not None:
                #add face to list of faces
                faces.append(face)
                #add label for this face
                labels.append(label)
    return faces, labels


In [19]:
print("Preparing data...")
faces, labels = prepare_training_data("training-data")
print("Data prepared")

#print total faces and labels
print("Total faces: ", len(faces))
print("Total labels: ", len(labels))


Preparing data...
Data prepared
Total faces:  3
Total labels:  3


In [20]:
#create our LBPH face recognizer 
face_recognizer = cv2.face.LBPHFaceRecognizer_create()


In [21]:
#train our face recognizer
face_recognizer.train(faces, np.array(labels))


In [22]:
def draw_rectangle(img, rect):
    (x, y, w, h) = rect
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
 
    #function to draw text on give image starting from
    #passed (x, y) coordinates. 
def draw_text(img, text, x, y):
    cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)


In [23]:
def predict(test_img):
    #make a copy of the image as we don't want to change original image
    img = test_img.copy()
    #detect face from the image
    face, rect = detect_face(img)

    #predict the image using our face recognizer 
    label = face_recognizer.predict(face)
    #get name of respective label returned by face recognizer
    label_text = subjects[label[0]]
 
    #draw a rectangle around face detected
    draw_rectangle(img, rect)
    #draw name of predicted person
    draw_text(img, label_text, rect[0], rect[1]-5)
    return img


In [24]:
print("Predicting images...")
# load test images
test_img1 = cv2.imread("test-data/1.jpg")
test_img2 = cv2.imread("test-data/2.jpg")
# perform a prediction
predicted_img1 = predict(test_img1)
predicted_img2 = predict(test_img2)
print("Prediction complete")

# display both images
cv2.imshow(subjects[1], predicted_img1)
cv2.imshow(subjects[2], predicted_img2)
cv2.waitKey(0)
cv2.destroyAllWindows()


Predicting images...
Prediction complete
